# Enunciado


3. Usando a experiência obtida na resolução dos problemas 1 e 2, e usando, ao invés  do grupo abeliano multiplicativo $\,\mathbb{F}_p^\ast\,$,  o  grupo abeliano aditivo que usou na pergunta 2,   
    1. Construa ambas as versões  IND-CPA segura e IND-CCA segura do esquema de cifra ElGamal em curvas elípticas.
    2. Construa uma implementação em curvas elípticas de um protocolo autenticado de “Oblivious Transfer” $\,\kappa$-out-of-$n\,$.

-----------

## Exercício 3.a

### **Versão IND-CPA**

#### Geração de chaves

**Chave Privada (sk)**: Um número inteiro aleatório no intervalo $[1, L-1]$, onde $L$ é a ordem do subgrupo gerado pelo ponto base $P$. \
**Chave Pública (pk)**: Um ponto na curva elíptica, calculado como $p_k = s_k * P$

#### Cifra

- Escolher um número inteiro aleatório $\kappa$ no intervalo $[1, L-1]$.
- Calcular o ponto $C_1 = \kappa * P$.
- Converter a mensagem $m$ num ponto na curva elíptica (como no método de Koblitz).
- Calcular o ponto $C_2 = m + \kappa * p_k$.
- O texto cifrado é o par $(C_1, C_2)$

#### Decifra

- Usar a chave privada $s_k$ para calcular $m = C_2 - s_k * C_1$.
- Converter o ponto $m$ de volta para a mensagem original

#### Diferenças entre usa o grupo multiplicativo $F_p^*$ e usar um grupo abeliano

No Exercício 3.A, a cifra e decifra não são iguais ao Exercício 1.A, porque as operações matemáticas mudam:

- No grupo multiplicativo $F_p^∗$, usamos exponenciação e multiplicação modular.
- Nas curvas elípticas, usamos adição de pontos.

Portanto, a cifra e decifra no Exercício 3.A devem ser adaptadas para as operações no grupo aditivo da curva elíptica. Comparando-as:

| Operação                     | Grupo Multiplicativo $ \mathbb{F}_p^* $ | Curva Elíptica $ E $ |
|------------------------------|-------------------------------------------|-------------------------|
| **Geração de Chaves (pk)**         | $ g^s\mod p $                          | $ g \cdot s $         |
| **Cifra**                     | $ \gamma = g^\omega \mod p $            | $ \gamma = \omega \cdot g $ |
|                              | $ \kappa = (g^s)^\omega \mod p $        | $ \kappa = \omega \cdot S $  |
|                              | $ \delta = m \cdot \kappa \mod p $      | $ C = M + \kappa $           |
| **Decifra**                   | $ \kappa = \gamma^s \mod p $            | $ \kappa = s \cdot \gamma $  |
|                              | $ m = \delta \cdot \kappa^{-1} \mod p $ | $ M = C - \kappa $           |

fonte: https://crypto.stackexchange.com/questions/9987/elgamal-with-elliptic-curves

#### Conversão de pontos

Especialmente no contexto de curvas elípticas, a conversão de pontos é uma técnica utilizada para mapear mensagens (ou dados) em pontos específicos sobre a curva elíptica.
Para conseguir agregar o melhor dos dois mundos (curvas elípticas e ElGamal) precisamos de mapear mensagens em pontos e vice-versa.

Nesta secção apresentamos a metodologia de conversão que iremos utilizar:

Para mapear uma mensagem de $p−1−l$ bits num ponto $(x,y)$ sobre uma curva elíptica definida no corpo primo abeliano de $p$ bits fazemos passo a passo:

1. **Preparação da Mensagem:**

    - A mensagem $m$ tem $p−1−l$ bits.
    - Para ajustar o tamanho, a mensagem é concatenada com $l$ bits zero, resultando em $x=m∥0l$.

2. **Cálculo de $x'$:**

    - O valor $x'$ é calculado como $x' = x^3 + a⋅x +b \mod p$, onde $a$ e $b$ são parâmetros da curva elíptica, e $p$ é o primo que define o corpo finito.

3. **Verificação do Resíduo Quadrático**:
    
    - Se $x'$ for um resíduo quadrático módulo $p$, então existe um $y$ tal que $y^2≡x' \mod p$. Nesse caso, o ponto $(x,y)$ é um ponto válido na curva.
    - Se $x'$ não for um resíduo quadrático, os últimos $l$ bits de $x$ são incrementados em 1, e o processo é repetido.

4. **Limite de Tentativas:**

    - O processo é repetido até $2^l$ vezes. Se nenhum $x'$ válido for encontrado, a mensagem é considerada "não codificável".

5. **Decodificação:**

    - Para recuperar a mensagem original, basta ignorar a coordenada $y$ e remover os últimos $l$ bits da coordenada $x$.

fonte: https://crypto.stackexchange.com/questions/76340/how-to-create-an-ec-point-from-a-plaintext-message-for-encryption

#### **Implementação**

Usando a classe definida no exercício 2:

In [1]:
import hashlib
import random
from sage.all import *

class EcDSA_Ed25519:
    def __init__(self, p, a, d):
        assert a != d and is_prime(p) and p > 3
        self.K = GF(p)  # Definido como atributo para possível uso futuro

        # Convertendo a e d para elementos do campo finito K
        self.a = self.K(a)
        self.d = self.K(d)

        # Calculando A e B dentro do campo finito K
        A = 2 * (self.a + self.d) / (self.a - self.d)
        B = 4 / (self.a - self.d)

        self.alfa = A / (3 * B)
        self.s = B

        # Calculando a4 e a6 no campo K
        a4 = self.s**(-2) - 3 * self.alfa**2
        a6 = -self.alfa**3 - a4 * self.alfa

        self.EC = EllipticCurve(self.K, [a4, a6])

        # Pontos base convertidos para K
        self.Px = self.K(15112221349535400772501151409588531511454012693041857206046113283949847762202)
        self.Py = self.K(46316835694926478169428394003475163141307993866256225615783033603165251855960)

        self.L = ZZ(2**252 + 27742317777372353535851937790883648493)
        self.P = self.ed2ec(self.Px, self.Py)

        self.private_key = self.generate_private_key()
        self.public_key = self.generate_public_key()

    def generate_private_key(self):
        return randint(1, self.L - 1)
    
    def generate_public_key(self):
        return self.private_key * self.P
    
    def ed2ec(self, x, y):
        if (x, y) == (0, 1):
            return self.EC(0)
        z = (1 + y) / (1 - y)
        w = z / x
        return self.EC(z / self.s + self.alfa, w / self.s)

def sign(e, message):
    h = hashlib.sha512(message).digest()
    h_int = int.from_bytes(h, 'big') % e.L
    
    while True:
        k = randint(1, e.L - 1)
        R = k * e.P

        r = int(R[0]) % e.L
        if r == 0:
            continue

        k_inv = pow(k, -1, e.L)
        s = (k_inv * (h_int + r * e.private_key)) % e.L

        if s == 0:
            continue

        return (r, s)

def verify(e, message, signature):
    r, s = signature

    if not (1 <= r < e.L and 1 <= s < e.L):
        return False
    
    h = hashlib.sha512(message).digest()
    h_int = int.from_bytes(h, 'big') % e.L

    k_inv = pow(s, -1, e.L)
    u1 = (h_int * k_inv) % e.L
    u2 = (r * k_inv) % e.L

    R_prime = u1 * e.P + u2 * e.public_key

    return int(R_prime[0]) % e.L == r

E definindo a cifra e decifra baseado no que foi abordado anteriormente:

In [2]:
import numpy as np

def gen_keys(E, q):
    '''
    Gerar chaves pública e privada de ElGamal para a cifra e decifra a partir da ordem q
    '''
    sk = randint(1, q - 1)
    pk = sk * E.P
    return sk, pk

def encrypt(E,message,public_key):
    '''
    Escolher um número inteiro aleatório k no intervalo [1, L-1].
    Calcular o ponto C1 = k * P.
    Converter a mensagem m num ponto na curva elíptica
    Calcular o ponto C2 = m + k * pk.
    O texto cifrado é o par (C1, C2)..
    '''
    k = randint(1, E.L - 1)
    C1 = k * E.P
    m_point = encode_message(E, message)
    C2 = m_point + k * public_key    
    return (C1, C2)
    
def decrypt(E,ciphertext,private_key):
    '''
    Usar a chave privada sk para calcular m = C2 - sk * C1.
    Converter o ponto m de volta para a mensagem original
    '''
    C1, C2 = ciphertext
    m_point = C2 - (private_key * C1)
    
    return decode_message(E, m_point)

def is_quadratic_residue(x, p):
    """
    Verifica se x é um resíduo quadrático módulo p.
    """
    return legendre_symbol(x, p) == 1

def encode_message(E, message, l=8):
    """
    Converte uma mensagem num ponto na curva elíptica.
    """
    K = E.K
    q = K.order()
    k = q.nbits() 
    m_int = int.from_bytes(message, 'big')
    m_bits = m_int.bit_length()

    # Ajusta o tamanho da mensagem para k-1-l bits
    if m_bits > k - 1 - l:
        raise ValueError("Mensagem muito grande para o campo finito.")

    # Concatena a mensagem com l bits zero
    x = (m_int << l)  # Equivalente a m || 0^l

    # Tenta encontrar um ponto válido na curva
    for i in range(2**l):
        x_prime = x + i  
        x_prime = K(x_prime)  

        # Calcula x' = x^3 + a*x + b (equação da curva)
        y_squared = x_prime**3 + E.EC.a4() * x_prime + E.EC.a6()

        # Verifica se y_squared é um resíduo quadrático
        if is_quadratic_residue(y_squared, q):
            y = y_squared.sqrt()
            return E.EC(x_prime, y) 
            
    raise ValueError("Não foi possível codificar a mensagem em um ponto da curva.")

def decode_message(E, point):
    """
    Converte um ponto na curva elíptica de volta para a mensagem original.
    """
    x = int(point[0]) 
    l = 8  
    m_int = x >> l 
    m_bytes = m_int.to_bytes((m_int.bit_length() + 7) // 8, 'big')
    return m_bytes

#### **Exemplo de uso:**

In [3]:
# Exemplo de uso
if __name__ == "__main__":
    # Parâmetros da curva Ed25519
    p = 2**255 - 19
    K = GF(p)
    a = K(-1)
    d = K(-121665) / K(121666)

    E = EcDSA_Ed25519(p, a, d)

    message = b"Hello, Ed25519!"

    try:
        m_point = encode_message(E, message)
        print(f"Mensagem codificada como ponto: {m_point}")
    except ValueError as e:
        print(e)

    decoded_message = decode_message(E, m_point)
    print(f"Mensagem decodificada: {decoded_message.decode()}")

    sk, pk = gen_keys(E, E.L)
    ciphertext = encrypt(E, message, pk)
    print(f"Texto cifrado: {ciphertext}")

    decrypted_message = decrypt(E, ciphertext, sk)
    print(f"Mensagem decifrada: {decrypted_message.decode()}")

    signature = sign(E,message)
    is_valid = verify(E,message, signature)
    print(f"Assinatura válida? {is_valid}")

Mensagem codificada como ponto: (96231036770510887582514965757268926721 : 9327398782287318265056166936429671725248326146653680186023877059085667908223 : 1)
Mensagem decodificada: Hello, Ed25519!
Texto cifrado: ((12796202878274455720422706475598825882405516446425762946628616373686619943941 : 10539011835253559098210635842697146524170200278425232715225734564315122175840 : 1), (10185570958684379981103720126097203433055520378810993774320897703052488232145 : 41111049837379051909010216971895143489305887735831403316931163705349961246383 : 1))
Mensagem decifrada: Hello, Ed25519!
Assinatura válida? True


------

### **Transformar um  PKE-IND-CPA em um PKE-IND-CCA**

A transformação FO original constrói, a partir de $\,(E_p,D_s)\,$,  um novo esquema de cifra assimétrica $\,(E'_p,D'_s)\,$ , usando um  “hash” pseudo-aleatório $\,h\,$ de tamanho $\,\lambda\,$ e um “hash” pseudo-aleatório $\,g\,$ de tamanho $\,|x|\,$.

O algoritmo de cifra parametrizado pelos dois “hashs”  $\,h,g\,$    é 

  $$E'_{p}(x)\;\equiv\;\vartheta\,r\gets \{0,1\}^\lambda\,\centerdot\,\vartheta\,y \gets x\oplus g(r)\,\centerdot\,\vartheta\,r'\gets h(r,y)\,\centerdot\,\vartheta\,c\gets f_p(r,r') \,\centerdot\, (y\,,\,c)$$

O algoritmo $\,D'_{s}\,$ rejeita o criptograma se detecta algum sinal de fraude. 


$$D'_{s}(y,c)\;\equiv\;\vartheta\,r \gets D_s(c)\,\centerdot\,\vartheta\,r'\gets h(r,y)\,\centerdot\,\mathsf{if}\;\;c\neq f_p(r,r')\;\;\mathsf{then}\;\;\bot\;\mathsf{else}\;\;y\oplus g(r)$$

### Implementação

In [4]:
def g(r,message):
    """Hash pseudoaleatório g(r) com tamanho igual ao da mensagem x"""
    g = hashlib.sha512()
    if isinstance(r, int):
        r_bytes = r.to_bytes((r.bit_length() + 7) // 8, 'big')
    else:
        r_bytes = r
    g.update(r_bytes)
    final_hash = g.digest()  
    while len(final_hash) < len(message):
        g = hashlib.sha512()
        g.update(r)
        final_hash += g.digest()
    return final_hash[:len(message)]

def h(r, y):
    """Hash pseudoaleatório h(r, y) com tamanho lambda_bits"""
    h = hashlib.sha512()
    if isinstance(r, int):
        r_bytes = r.to_bytes((r.bit_length() + 7) // 8, 'big')
    else:
        r_bytes = r
    ry = bytes(a ^^ b for a, b in zip(r_bytes, y))
    h.update(ry)
    return int.from_bytes(h.digest()[:8], 'big')

In [5]:
def f_p(E,public_key,r,rlinha):
    C1 = rlinha * E.P
    if isinstance(r,int):
        r_point = encode_message(E, r.to_bytes((r.bit_length() + 7) // 8, 'big'))
    else:
        r_point = encode_message(E, r)
    C2 = r_point + rlinha * public_key
    return (C1, C2)

In [6]:
def encrypt_FO(E, message, public_key):
    max_bits = E.K.order().nbits() - 1 - 8  # k = 255, l = 8 → 246 bits
    r = randint(1, 2**max_bits - 1)
    print("r (cifra):", r)
    
    y = bytes(a ^^ b for a, b in zip(message, g(r, message)))
    rlinha = h(r, y)
    c = f_p(E, public_key, r, rlinha)
    return (y, c)

def decrypt_FO(E, ciphertext, public_key, private_key):
    y, c = ciphertext
    C1, C2 = c
    
    r = decrypt(E,c, private_key)
    print("r (decifra):", r)
    
    rlinha = h(r, y)
    
    if c != f_p(E,public_key,r,rlinha):
        raise ValueError("ABSURDO")
    
    res = bytes(a ^^ b for a, b in zip(y, g(r, y)))
    return res

In [7]:
message = b"Ola!!!!!!!!"
try:
    m_point = encode_message(E, message)
    print(f"Mensagem codificada como ponto: {m_point}")
except ValueError as e:
    print(e)

decoded_message = decode_message(E, m_point)
print(f"Mensagem decodificada: {decoded_message.decode()}")

sk, pk = gen_keys(E, E.L)
ciphertext = encrypt_FO(E, message, pk) 
print(f"Texto cifrado: {ciphertext}")

decrypted_message = decrypt_FO(E, ciphertext, pk, sk)
print(f"Mensagem decifrada: {decrypted_message.decode()}")

signature = sign(E,message)
is_valid = verify(E,message, signature)
print(f"Assinatura válida? {is_valid}")

Mensagem codificada como ponto: (24580338445083258871783432449 : 6780190025506922750797619835010078147534407628739436173625509384659206151888 : 1)
Mensagem decodificada: Ola!!!!!!!!
r (cifra): 111038085910335465676998667114447207859806939094428397776477970140073371180
Texto cifrado: (b'\xab_\xbc\xd4m\x9av\xd7\xa8\x12\xa8', ((16584393027713842232139056897724912841051841278146423941962458733550268346437 : 17743273331963980743891381743316976663123145041775440794235814248971444004121 : 1), (5140329393361717998424713553212117889306563811055912485028548595432542357456 : 23299085056816125830057292786804669373978206241499866185736023708930732067324 : 1)))
r (decifra): b'>\xd8g\x83\x1e\xff\x8e\xe5\xf1\xba\x12\xa4\xe2b\xad\x14\x16\xa0S\xe8\xd5\xdd\xcf\x1d\xe0kH\x1d\x91&,'
Mensagem decifrada: Ola!!!!!!!!
Assinatura válida? True


##  Exercício 3.b

### “Oblivious Transfer” $\,\kappa$-out-of-$n\,$


O protocolo de “oblivious transfer”  implementa um mecanismo de transferência de informação entre dois agentes: o **Provider** (também designado por Sender) e o **Receiver** (também designado por Adversário) . Em linhas gerais, o protocolo caracteriza-se da forma seguinte:


1. O **Provider** põe à disposição para comunicação futura $\,n\,$ items de informação (ou mensagens) que ele enumera como $\,m_1, m_2, \cdots,m_{n}\;$ e que armazena de forma privada. 
    Nesta fase a única informação tornada pública é o número  de mensagens  $\,n$ .
2. O Receiver informa o **Provider** que pretende receber $\,\kappa\,$ das $\,n\,$ mensagens
3. Caso o **Provider** aceite o par $(n,\kappa)\;$ os dois agentes, a começar pelo **Provider**, trocam uma sequência de mensagens e, no final,
    1. O **Receiver** passa a conhecer exatamente  $\,\kappa\,$ mensagens mas continua a ignorar o conteúdo de todas as restantes $\,n-\kappa\,$ mensagens.
    2. O **Provider** ignora a identificação (“is oblivious of”)  das $\,\kappa\,$ mensagens que o Receiver passou a conhecer.

O protocolo usa um esquema PKE $\,\{(E_p,D_s)\}_{(s,p)\in\mathcal{G}}$ que neste caso irão ser a cifra e decifra do exerício 3.a.. 

### Criterion

In [8]:
import numpy as np
from sage.schemes.elliptic_curves.ell_point import EllipticCurvePoint_field
class CknCriterion:
    def __init__(self, kappa, n, E):
        self.kappa = kappa
        self.n = n
        self.q = E.L
        self.seed = np.random.randint(0, 2**32)
        self.A = self.generate_A()
        self.u = self.generate_u()
        self.Fp = GF(self.q).unit_group()
    
    def generate_A(self):
        """Gera a matriz A usando XOF a partir da seed"""
        np.random.seed(self.seed)
        A = random_matrix(GF(self.q), self.n, self.n - self.kappa)
        return A
    
    def generate_u(self):
        """Gera o vetor u, que deve ser não nulo"""
        np.random.seed(self.seed + 1) 
        u = vector(GF(self.q), [randint(1, self.q - 1) for _ in range(self.n - self.kappa)])
        return u
    
    def verify(self, p):
        """Verifica se p satisfaz o critério Ckn, ou seja, se p * A = u"""
        if len(p) != self.n:
            raise ValueError(f"p deve ter {self.n} elementos")
        
        p_values = []
        for x in p:
            if isinstance(x, EllipticCurvePoint_field):  
                p_values.append(int(x[0]))
            else:
                p_values.append(int(x))  
                
        Zq = GF(self.q)
        p_vector = vector(Zq, p_values)
        
        A_matrix = matrix(Zq, self.A) 
        pA = p_vector * A_matrix
        
        u_vector = vector(Zq, self.u)  
        return pA == u_vector
    
    def print_criterion(self):
        print(f"Matriz A:\n{self.A}")
        print(f"Vetor u:\n{self.u}")

### Provider

In [9]:
class Provider:
    def __init__(self, pk, sk, n_mensagens,E):
        self.pk,self.sk = gen_keys(E,E.L)
        self.numero_de_mensagens = n_mensagens
        # Informação privada das mensagens:
        self.messages = [f"mensagem{i}" for i in range(n_mensagens)]
        self.criterion = None
    def define_criterion(self,kappa):
        n = self.numero_de_mensagens
        q = pk[1]
        self.criterion = CknCriterion(kappa, n, E)

In [10]:
n_mensagens = 100
# Parâmetros da curva Ed25519
p = 2**255 - 19
K = GF(p)
a = K(-1)
d = K(-121665) / K(121666)

E = EcDSA_Ed25519(p, a, d)

provider = Provider(None,None,n_mensagens,E)
print("self.pk,self.sk",provider.pk,provider.sk)
print("self.numero_de_mensagens",provider.numero_de_mensagens)
print("self.messages",provider.messages)
print("self.criterion = None")

self.pk,self.sk 6858816691447425038548754815709884695392465002284498898052289533587584370316 (3060208343119044260941447008748772866118330713052370879265828165288479340433 : 56729721669481270997740425657821911750968915534023101580171855114596804815157 : 1)
self.numero_de_mensagens 100
self.messages ['mensagem0', 'mensagem1', 'mensagem2', 'mensagem3', 'mensagem4', 'mensagem5', 'mensagem6', 'mensagem7', 'mensagem8', 'mensagem9', 'mensagem10', 'mensagem11', 'mensagem12', 'mensagem13', 'mensagem14', 'mensagem15', 'mensagem16', 'mensagem17', 'mensagem18', 'mensagem19', 'mensagem20', 'mensagem21', 'mensagem22', 'mensagem23', 'mensagem24', 'mensagem25', 'mensagem26', 'mensagem27', 'mensagem28', 'mensagem29', 'mensagem30', 'mensagem31', 'mensagem32', 'mensagem33', 'mensagem34', 'mensagem35', 'mensagem36', 'mensagem37', 'mensagem38', 'mensagem39', 'mensagem40', 'mensagem41', 'mensagem42', 'mensagem43', 'mensagem44', 'mensagem45', 'mensagem46', 'mensagem47', 'mensagem48', 'mensagem49', 'mensagem5

### Receiver

In [11]:
import hashlib
from random import sample

class Receiver:
    def __init__(self, k, n_mensagens, q, E):
        self.k = k  # Número de chaves privadas geradas
        self.n_mensagens = n_mensagens
        self.q = E.L
        self.I = sample(range(n_mensagens), k)  # Seleção aleatória de k índices
        self.e = self.enumeration(self.I)  
        self.p,self.s_values = self.generate_keys()  
        self.s = self.generate_secret()
        self.tau = self.generate_authentication_tag()

    def enumeration(self, I):
        """Cria a função de enumeração que mapeia {1, 2, ..., κ} para os elementos de I ordenados"""
        I_sorted = sorted(I)
        return {i + 1: I_sorted[i] for i in range(len(I_sorted))}

    def generate_secret(self):
        """Gera um segredo aleatório (simulado como um número grande)"""
        return ZZ.random_element(2**32)

    def generate_keys(self):
        """Gera κ chaves privadas e publicas usando o genkeys de ElGamal com curvas elípticas"""
        vetor_pk = [0] * self.n_mensagens 
        vetor_sk = []
        for i in range(1,self.k+1):
            sk,pk = gen_keys(E,self.q)
            vetor_sk.append(sk)
            vetor_pk[self.e[i]] = pk
        return vetor_pk,vetor_sk

    def generate_authentication_tag(self):
        """Gera a tag de autenticação hash(I, s)"""
        data = str(self.I) + str(self.s)
        return hashlib.sha256(data.encode()).digest()

    def complete_p_vector(self, A, u):
        """Completa o vetor p para satisfazer p * A = u no corpo finito Z_q."""
        Zq = GF(self.q)  # Define o corpo finito Z_q
        
        # Identificar os índices já preenchidos (valores diferentes de 0)
        filled_indices = [i for i in range(self.n_mensagens) if self.p[i] != 0]
        filled_values = vector(Zq, [int(self.p[i][0]) if isinstance(self.p[i], EllipticCurvePoint_field) else int(self.p[i]) for i in filled_indices])
        
        # Criar a matriz A_filled (linhas correspondentes aos índices preenchidos)
        A_filled = matrix(Zq, [A[i] for i in filled_indices])
        
        # Criar a matriz A_empty (linhas correspondentes aos índices vazios, ou seja, onde p[i] == 0)
        A_empty = matrix(Zq, [A[i] for i in range(A.nrows()) if i not in filled_indices])
        
        # Calcular u' = u - (filled_values * A_filled)
        u_prime = vector(Zq, u) - filled_values * A_filled
        
        # Resolver o sistema linear A_empty^T * p_empty = u' no corpo finito Z_q
        try:
            A_empty_T = A_empty.transpose()
            p_empty = A_empty_T.solve_right(u_prime)
        except:
            # Se o sistema for singular, tentar solução alternativa (ex: mínimos quadrados)
            p_empty = A_empty_T.pseudoinverse() * u_prime
        
        # Preencher os elementos desconhecidos no vetor p (apenas onde p[i] == 0)
        empty_indices = [i for i in range(self.n_mensagens) if self.p[i] == 0]
        for i, idx in enumerate(empty_indices):
            # Gerar uma chave pública "má" usando gen_keys
            sk_mau, pk_mau = gen_keys(E, self.q)
            self.p[idx] = pk_mau
        
        return self.p
    
    def print_info(self):
        print("----------------------------------------------------------------------")
        print(f"Seleção I: {self.I}")
        print("----------------------------------------------------------------------")
        print(f"Função de enumeração e: {self.e}")
        print("----------------------------------------------------------------------")
        print(f"Segredo s: {self.s}")
        print("----------------------------------------------------------------------")
        print(f"Chaves privadas s_i: {self.s_values}")
        print("----------------------------------------------------------------------")
        print(f"Vetor p (com chaves públicas mapeadas): {self.p}")
        print("----------------------------------------------------------------------")
        print(f"Tag de autenticação τ: {self.tau}")
        print("----------------------------------------------------------------------")

In [12]:
k = 20
receiver = Receiver(k,n_mensagens,provider.pk,E) # Receiver escolhe o conjunto I já na sua construção

----------

Definidas as classes seguimos os passos igualmente como no exercício 1:

### 1. O **Provider** gera o critério C_{k,n} e envia-o ao **Receiver**

In [13]:
provider.define_criterion(k)

### 2. O Receiver  escolhe um conjunto $\,I \subset \{1,n\}\,$ , de tamanho $\,\#I = \kappa\,$ , que identifica os índices das mensagens que pretende recolher.  
Seja $\;e\;$ a enumeração de $\,I\,$:  a função crescente $\,e\colon\{1,\kappa\}\to \{1,n\}\;$  cuja imagem é $\,I\,$.

O Receiver compromete-se com a escolha de mensagens da seguinte forma (dado o conjunto $I$ e a função crescente $e$):

1. Gera aleatoriamente um segredo $\,\mathbf{s}\,$ e , usando um XOF com$\,\mathbf{s}\,$ como “seed”, constrói$\,\kappa\,$  chaves privadas$\,s_1,\cdots,s_\kappa\,$ .
2. Para cada$\,i\in \{1,\kappa\}\,$ gera chaves públicas $\;\upsilon_i \gets \mathsf{pk}(s_i)\,$ e atribui o valor$\,\upsilon_i\,$ à componente de ordem$\,e(i)\,$ do vector $\,\mathsf{p}\;$; ou seja , executa$\,$ $\,\mathsf{p}_{e(i)} \gets \upsilon_i\,$
3. Gera uma “tag” de autenticação$\,$para a seleção$\,I\,$ e o segredo$\,\mathbf{s}\,$

$$\,\tau \gets \mathsf{hash}(I\,,\,\mathbf{s})\,$$

(Definido na classe:)

In [14]:
receiver.print_info()

----------------------------------------------------------------------
Seleção I: [50, 47, 83, 80, 60, 40, 91, 7, 10, 42, 75, 33, 45, 59, 92, 37, 65, 67, 28, 88]
----------------------------------------------------------------------
Função de enumeração e: {1: 7, 2: 10, 3: 28, 4: 33, 5: 37, 6: 40, 7: 42, 8: 45, 9: 47, 10: 50, 11: 59, 12: 60, 13: 65, 14: 67, 15: 75, 16: 80, 17: 83, 18: 88, 19: 91, 20: 92}
----------------------------------------------------------------------
Segredo s: 2503999058
----------------------------------------------------------------------
Chaves privadas s_i: [5948516612425386760688962138372166375415731937920739830488629195468777516288, 5931750751854221369200472110127535193033528484910412540102493947049429823102, 6837120425531107983606787080546193706679477441345741980032027025796559859750, 3539612607525788996726565258349321003537409593331303339969493545620178039831, 2745134187821038079575708328460235617273866533207458379327576948614195441687, 4000394779761075

Em seguida completa a definição de $\,\mathsf{p}\,$ atribuindo às compompentes $\,\{\mathsf{p}_j\}_{j\in\!\!\!/I}$valores tais que o vetor de chaves públicas  $\,\mathsf{p}\,$ seja aceite pelo critério $\,\mathcal{C}_{\kappa,n}\,$.

Para que o vetor **p** satisfaça a equação $p×A=u$, onde **A** é a matriz gerada pelo critério $C_{k,n}$ e **u** é o vetor correspondente, o Receiver precisa preencher os espaços em **p** que não foram preenchidos pelas chaves públicas de forma que a equação seja satisfeita.
(Explicação mais a fundo no exercício 1)

In [15]:
completed_p = receiver.complete_p_vector(provider.criterion.A, provider.criterion.u)
tau = receiver.tau

### 3. O Provider  determina  $\,\mathcal{C}_{\kappa,n}(\mathsf{p})\,$; se $\,\mathsf{p}\,$ não for aceite pelo critério então aborta o protocolo.

Se $\,\mathsf{p}\,$  for aceite,  então usa  a variante IND-CCA  da cifra IND-CPA com a “tag”  $\tau$

$$E'_{p}(x,\tau)\;\equiv\;\vartheta\,r\gets \{0,1\}^\lambda\,\centerdot\,\vartheta\,y \gets x\oplus g(r)\,\centerdot\,\vartheta\,r'\gets h(r,y,\tau)\,\centerdot\,\vartheta\,c\gets f_p(r,r') \,\centerdot\, (y\,,\,c)$$

cuja característica específica é o facto de se incluir o “tag” $\,\tau\,$ no “hash”  $\;h(r,y,\tau)\;$usado para construir a pseudo-aleatoriedade $\,r'\,$.

Usando esta cifra o Provider  constrói $\,n\,$ criptogramas 
$$\;(y_i,c_i) \gets E'_{p_i}(m_i)\quad$$  com $\,i\in\{1,n\}$
que envia para o Receiver.

Iremos definir um novo "hash" pseudo-aleatório $h$ para que consiga receber como argumento a tag $\tau$:

In [16]:
def h_OT(r,y,t):
    """Hash pseudoaleatório h(r, y, t)"""
    h = hashlib.sha512()
    if isinstance(r,int):
        r_bytes = r.to_bytes((r.bit_length() + 7) // 8, 'big')
    else:
        r_bytes = r
    ry = bytes(a ^^ b for a, b in zip(r_bytes, y))
    ryt = bytes(a ^^ b for a, b in zip(ry, t))
    h.update(ryt)
    #full_hash = h.digest()[:16]
    return int.from_bytes(h.digest()[:8],'big')

E definir a cifra, que na realidade irá ser a do exercício 1.b. mas com um argumento extra $\tau$:

In [17]:
def f_p_OT(E,public_key,r,rlinha):
    C1 = rlinha * E.P
    if isinstance(r,int):
        r_point = encode_message(E, r.to_bytes((r.bit_length() + 7) // 8, 'big'))
    else:
        r_point = encode_message(E, r)
    C2 = r_point + rlinha * public_key
    return (C1, C2)

def encrypt_FO_OT(E, message, public_key,tau):
    # Gerar r com tamanho máximo de 246 bits (k-1-l)
    max_bits = E.K.order().nbits() - 1 - 8  # k = 255, l = 8 → 246 bits
    r = randint(1, 2**max_bits - 1)
    print("r (cifra):", r)
    
    # Resto do código permanece igual
    y = bytes(a ^^ b for a, b in zip(message, g(r, message)))
    rlinha = h_OT(r, y,tau)
    c = f_p_OT(E, public_key, r, rlinha)
    return (y, c)

Determinamos então $C_{k,n}(p)$, se este for aceite ciframos todas as mensagens com as chaves públicas fornecidas em $p$:

In [18]:
import sys
from sympy import isprime
from sage.schemes.elliptic_curves.ell_point import EllipticCurvePoint_field  # Importação necessária

if not provider.criterion.verify(completed_p):
    print("------------------------------------------------------------------------")
    print("O vetor p foi aceite")
    print("p final:[ ")
    for public_key in completed_p:
        print(public_key)
    print("]")
    print("------------------------------------------------------------------------")
    ciphertext_vector = []
    i = 0
    for public_key in completed_p:
        if isinstance(public_key, EllipticCurvePoint_field):  # Verificação correta
            byte_length = (sys.getsizeof(provider.messages[i]))
            ciphertext_vector.append(encrypt_FO_OT(E, bytes(provider.messages[i], 'utf-8'), public_key, tau))
            i += 1
        else:
            raise ValueError(f"encontrada chave pública inválida: {public_key}")
else:
    print("O vetor p não foi aceite. Abortado")

------------------------------------------------------------------------
O vetor p foi aceite
p final:[ 
(38946869806052897080837151464228836528685444687240191834047029992288277149848 : 32691426237203135565371095367731027902290292352654239767940499671478851873252 : 1)
(47562965850066009622480877046204029877852590107443135639614308069063048943623 : 38190748760462678362783017850183367754404633333296170490178085625179306610864 : 1)
(25957734988990609026096591906542223167191345417797152184359886576793534029867 : 35167027615009104114133762153386679508111201419111028147205225946687649704654 : 1)
(55318099651296852351136369732904229855829055345505842015370346190573295028512 : 3711022813268273006031889010338561795135173044723965761776613085147830675691 : 1)
(13839394646689273462878478650787124404016966891287640061912670854489199662080 : 54557458817180840092090139319101909568430391699632271586900027714462443619389 : 1)
(155539043705271737553818895740767277002386267236788037329025050737068193696

### 4. O Receiver  usa  a variante IND-CCA  da cifra IND-CPA com a “tag” de autenticação $\tau$
$$D'_{s}(y,c,\tau)\;\equiv\;\vartheta\,r \gets D_s(c)\,\centerdot\,\vartheta\,r'\gets h(r,y,\tau)\,\centerdot\,\mathsf{if}\;\;c\neq f_p(r,r')\;\;\mathsf{then}\;\;\bot\;\mathsf{else}\;\;y\oplus g(r)$$
         
uma vez mais a única característica particular deste algoritmo é o uso da “tag” de autenticação $\,\tau\,$ na construção da pseudo-aleatoriedade $\;r'\gets h(r,y,\tau)\;$.


O agente Receiver 
   - conhece, porque criou,  a “tag” $\,\tau\,$que autentica o conjunto de mensagens escolhidas $\,I\,$ e  o respetivo conjunto de chaves públicas (as “boas” chaves).
    - conhece , porque gerou e armazenou num passo anterior,  as chaves privadas $\,s_i\,$ para todos $\,i\in I\,$  
    - conhece, porque recebeu do Receiver,  todos os criptogramas $\{(y_i,c_i)\}_{i\in\{1,n\}}\,$
        
Então,  para todo $\,i\in I\,$,  pode recuperar  a mensagem
                                                        $$m_i \,\gets\, D_{s_i}(y_i,c_i,\tau)$$ 


Como na cifra vamos fazer com que a decifra possa receber a tag de autenticação:

In [19]:
def decrypt_FO(E, ciphertext, public_key, private_key,tau):
    y, c = ciphertext
    C1, C2 = c
    
    r = decrypt(E,c, private_key)
    
    rlinha = h_OT(r, y, tau)
    
    if c != f_p(E,public_key,r,rlinha):
        raise ValueError("ABSURDO")
    
    res = bytes(a ^^ b for a, b in zip(y, g(r, y)))
    return res.decode('utf-8')

In [20]:
# Decifrar e verificar as mensagens
decrypted_messages = []
print(len(ciphertext_vector))
for msg_number, idx in receiver.e.items(): 
    if idx < len(ciphertext_vector):  # Verifica se o índice é válido
        print(f"\nNúmero da mensagem: {msg_number}")
        print(f"Índice no vetor ciphertext_vector: {idx}")
        print(f"Chave privada (sk): {receiver.s_values[msg_number - 1]}")  # Ajuste para índice base 0
        print(f"Chave pública (pk): {receiver.p[idx]}")
        # Verifica se a mensagem foi cifrada corretamente
        if ciphertext_vector[idx] is None:
            print("idx:",idx)
            print(f"Mensagem {idx} não foi cifrada corretamente.")
            continue

        y, c = ciphertext_vector[idx]
        sk = receiver.s_values[msg_number - 1] 
        decrypted_message = decrypt_FO(E, (y,c), receiver.p[idx],sk, tau)

        if decrypted_message is not None:
            print(f"Mensagem decifrada: {decrypted_message}")

            # Compara com a mensagem original no Provider
            if decrypted_message == provider.messages[idx]:
                print("Decifração bem-sucedida! A mensagem decifrada corresponde à original.")
                decrypted_messages.append(decrypted_message)
            else:
                print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                print("Erro na decifração: A mensagem decifrada NÃO corresponde à original.")
                print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        else:
            print("Erro na decifração intencional.")

100

Número da mensagem: 1
Índice no vetor ciphertext_vector: 7
Chave privada (sk): 5948516612425386760688962138372166375415731937920739830488629195468777516288
Chave pública (pk): (37941807582977062815170480695850709414656908904139673194061162179589851867622 : 51376412430721312089102854720836315511483286743892982202114681177606793478677 : 1)
Mensagem decifrada: mensagem7
Decifração bem-sucedida! A mensagem decifrada corresponde à original.

Número da mensagem: 2
Índice no vetor ciphertext_vector: 10
Chave privada (sk): 5931750751854221369200472110127535193033528484910412540102493947049429823102
Chave pública (pk): (49533706615044597912057931493748257275810045026757082190148411422891173707556 : 23517599525967663317788036073944149245142979661197414840967271847055430840190 : 1)
Mensagem decifrada: mensagem10
Decifração bem-sucedida! A mensagem decifrada corresponde à original.

Número da mensagem: 3
Índice no vetor ciphertext_vector: 28
Chave privada (sk): 6837120425531107983606787080546